## Color compression

We now discuss the Gif system which is powerfull to compress images. Actualy, it also interessant, because the central algorithm (the kmeans) can be very useful in many situations.

### Histogram and quantization

In [ ]:
def histogramme(image):
    hist = np.zeros(256)
    for j in range(image.shape[0]):
        for i in range(image.shape[1]):
            valeur = image[j, i]
            hist[valeur] += 1
    return hist

***To you:***  $(2\heartsuit)$ Rewrite the previous function without loop. Help: use `np.unique` with the good arguments.


In [ ]:
img = img_origin.copy()
red = img[:, :, 0]
green = img[:, :, 1]
blue = img[:, :, 2]

h_red = histogramme(red)
h_green = histogramme(green)
h_blue = histogramme(blue)
maxi=max([h_red.max(),h_blue.max(),h_blue.max()])
plt.figure(figsize=(8, 6))
plt.plot(h_red,c='red')
plt.plot(h_green,c='green')
plt.plot(h_blue,c='blue')
plt.axis([-2, 257, 0, maxi])
plt.xlabel("valeur")
plt.ylabel("Nombre");

Such histograms are often showed in computer softwares which allow you to modify photos.

***To you:***

* $(1\heartsuit)$ Explain the pics of the histograms at 255
* $(1\heartsuit)$ Explain the two montains blue and green around 20.


In [ ]:
img_quant = (img//10 )*10

plt.figure()
plt.plot(histogramme(img_quant[:, :, 0]), c='red')
plt.plot(histogramme(img_quant[:, :, 1]), c='green')
plt.plot(histogramme(img_quant[:, :, 2]), c='blue')
plt.xlabel("valeur")
plt.ylabel("Nombre")


imageio.imwrite("out/babouin_quantized.png", img_quant)

***To you:***

* $(1\heartsuit)$ Explain what do the operation `(img//10 )*10`
* $(1\heartsuit)$ How many different colors can be present in `img_quant`
* $(1\heartsuit)$ How such operation can be used to compress the image

In [ ]:
IPython.display.Image("out/babouin_quantized.png")

Above, we see that the quantization does not deteriorate the visual aspect.

### Gif principle

We have seen that we can drasticaly decrease the number of colors without deteriorate the image. In the previous quantization, we allowed to  (r,g,b) to range into the cube $\{0,...,25\}\times\{0,...,25\}\times \{0,...,25\}$.

But it is actually not obligatory to chose a cube as a set of possible colors. We can also select a list of colors, and attribuate to each pixel one of them.

So we have to select a short list of point of $\mathbb N^3$ which represent all the colors of the original image. This is the Gif principle.

The algorithm to chose the list of represents is called the K-means. It also has many other application in many domain (see later on).  

### K-means




K-means is an algorithm to gather points by clusters. We have to chose at the very begin the number of cluster. Let's say 3 for the example. Then the algorithm produce 3 centroid which are central inside each cluster. Observe:


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
"""some points (do not look the way we generate them)"""
points=np.concatenate([np.random.normal(size=[100,2]),np.random.normal(size=[130,2])+np.array([4,1])])
plt.plot(points[:,0],points[:,1],"+");

The methode `fit()`  compute the centroids. They are register in the attribute `.cluster_centers_`:

In [ ]:
"""creation of the algo"""
kmeans=KMeans(n_clusters=3)
"""computation of centroids"""
kmeans.fit(points)
"""here is the result"""
kmeans.cluster_centers_

In [ ]:
"""let's plot points and centroid"""
plt.plot(points[:,0],points[:,1],"+");
for centroid in kmeans.cluster_centers_:
    plt.plot(centroid[0],centroid[1],"o")


Now, to separate points into cluster, we just associate each point to its closest centroid. This is made by the method `predict()`, which produce a list giving the indexes of the closest centroids.

In [ ]:
labels = kmeans.predict(points)
labels

In [ ]:
plt.scatter(points[:,0],points[:,1],c=labels,marker="+")
for centroid in kmeans.cluster_centers_:
    plt.plot(centroid[0],centroid[1],"o")

***Remark:*** Probably for this points, only 2 centroids would produce a better segmentations of the points. But the kmeans algorithm is not capable to decide how many clusters is the best choice.


Actualy, the objet `kmean` can also associate other points to the previous centroids:

In [ ]:
"""we generate some points regularly placed"""
x=np.linspace(-2,6,20)
XX,YY=np.meshgrid(x,x)
XX=XX.flatten()
YY=YY.flatten()
other_points=np.stack([XX,YY],axis=1)
other_points.shape

In [ ]:
plt.plot(other_points[:,0],other_points[:,1],'+');

In [ ]:
labels = kmeans.predict(other_points)
labels

In [ ]:
plt.scatter(other_points[:,0],other_points[:,1],c=labels,marker="+")
for centroid in kmeans.cluster_centers_:
    plt.plot(centroid[0],centroid[1],"o");

### How its work?

The kmeans algorithm is so important and so simple, that you must understand it.


* At the begining, it place the 3 centroid at random. The simpler way is to chose 3 positions among the position of points (so that the centroid are not too far from the points).  
* Then we produce 3 clusters by associating points to their closest centroid.
* Then centroid are moved at the barycenter of the clusters.
* Then new clusters are computed
* Then new barycenter are computed
* And so on, until the centroid position stabilize itself

***To you:*** Try this on a sheet of paper: that's work very well.


Now it could be a very interesting exercise to implement it yourself. But do this only if you have finished to read the whole chapter.


An important function for kmeans is the fonction which associate points to their closest centroid. This function is given below.

***To you:*** Code the complete algorithm. You can make class for this (as `sklearn`), but it is not realy necessary (make as you want).

In [ ]:
def findClosestCentroid(points, centroids):
    """  allPairwiseDiff[i,j,k]= points[j,k]-centroids[i,k] """
    allPairwiseDiff =  points[np.newaxis,:,:]-centroids[:,np.newaxis,:]
    squareDist =  allPairwiseDiff[:,:,0]**2+allPairwiseDiff[:,:,1]**2
    return np.argmin(squareDist, axis=0)

In [ ]:
centroids = np.array([[1.,1],[-1,5],[5,0],[5,4]])
labels=findClosestCentroid(other_points,centroids)
labels

In [ ]:
plt.scatter(other_points[:,0],other_points[:,1],c=labels,marker="+")
for centroid in centroids:
    plt.plot(centroid[0],centroid[1],"o")

***To you:*** $(1\heartsuit)$ Do you now the name of the colored cells above?

### Gif example

In [ ]:
"""a very small number of colors"""
n_colors = 10
""" we work with float in [0,1] instead of integers """
img = img_origin.astype(np.float64)/255

In [ ]:
h,w,d=img.shape
assert d==3
pixels=img.reshape([h*w,3])
pixels.shape

In [ ]:
"""a random selection of pixel"""
pixel_sample = pixels[np.random.permutation(h*w)[:5000]]
"""the kmeans algo will produce centroids which represent the sample"""
kmeans = KMeans(n_clusters=n_colors).fit(pixel_sample)

In [ ]:
""" the labels which associate pixels to centroids """
labels = kmeans.predict(pixels)

In [ ]:
def recreate_image(codebook, labels, h,w):
    """Recreate the (compressed) image from the code book & labels"""
    image = np.zeros((h, w, 3))
    label_idx = 0
    for i in range(h):
        for j in range(w):
            image[i,j] = codebook[labels[label_idx]]
            label_idx += 1
    return image

In [ ]:
plt.title('Quantized image')
plt.imshow(recreate_image(kmeans.cluster_centers_, labels, h, w));

***To you:*** $(1\heartsuit)$ How many colors at minimum need our image to be nice (find this number by testing).

### An other application of kmeans


skip this part if you are late.

With some random procedure, it is very easy to put many points on a manifold (ex: a sphere). But if you want to produce a mesh of this manifold, you need to have not too much points, but they need to be regularly distributed.

***To you:***  $(10\heartsuit)$ Make a function which pick up n-points regularly distibuted on the unit sphere in $\mathbb R^d$. Make graphical illustration for $d=2$ and $d=3$.


